In [15]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

## TFIDF

In [16]:
train_data = pd.read_pickle(open("tfidf/train_tfidf.pkl", "rb"))
train_X = list(train_data['TFIDF'])
train_y = train_data['Sentiment']
dev_data = pd.read_pickle(open("tfidf/dev_tfidf.pkl", "rb"))
dev_X = list(dev_data['TFIDF'])
dev_y = dev_data['Sentiment']

In [17]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_X, train_y)
labels_predict = knn.predict(dev_X)
labels_predict

array(['negative', 'positive', 'negative', ..., 'positive', 'positive',
       'negative'], dtype=object)

In [18]:
print(classification_report(dev_y, labels_predict))
print(confusion_matrix(dev_y, labels_predict))

              precision    recall  f1-score   support

    negative       0.60      0.63      0.62      2000
    positive       0.61      0.59      0.60      2000

    accuracy                           0.61      4000
   macro avg       0.61      0.61      0.61      4000
weighted avg       0.61      0.61      0.61      4000

[[1253  747]
 [ 821 1179]]


## Embedding

In [19]:
train_data = pd.read_pickle(open("sentence-transformers/train_emb.pkl", "rb"))
train_X = list(train_data['TFIDF'])
train_y = train_data['Sentiment']
dev_data = pd.read_pickle(open("sentence-transformers/dev_emb.pkl", "rb"))
dev_X = list(dev_data['TFIDF'])
dev_y = dev_data['Sentiment']

In [20]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_X, train_y)
labels_predict = knn.predict(dev_X)
labels_predict

array(['negative', 'positive', 'negative', ..., 'positive', 'positive',
       'negative'], dtype=object)

In [21]:
print(classification_report(dev_y, labels_predict))
print(confusion_matrix(dev_y, labels_predict))

              precision    recall  f1-score   support

    negative       0.66      0.66      0.66      2000
    positive       0.66      0.67      0.66      2000

    accuracy                           0.66      4000
   macro avg       0.66      0.66      0.66      4000
weighted avg       0.66      0.66      0.66      4000

[[1318  682]
 [ 667 1333]]
